Import Modules

In [1]:
import pandas as pd 
import numpy as np

load datasets 

In [ ]:
#Load COI data
coi_data = pd.read_csv('2_subdomains.csv', usecols=[
    'county_name', 'state_name',  # Geographic identifiers for merging
    
    #if we're going to do stepwise feature selection, might as well pull as many relevant features as possible then omit during testing
    
    
    # Education Domain
    'z_ED_EC_nat',  # Early Childhood Education: Access to and quality of early childhood education programs.
    'z_ED_EL_nat',  # Elementary Education Quality: Performance and access at the elementary school level.
    'z_ED_ER_nat',  # High School Enrollment Rates: Percentage of eligible students enrolled in high school.
    'z_ED_SP_nat',  # School Performance: Aggregate performance of schools in the area, indicating overall education quality.
    
    # Health & Environment Domain
    'z_HE_EP_nat',  # Environmental Pollution: Air and water pollution levels, which impact chronic health conditions.
    'z_HE_HR_nat',  # Housing Risk: Measures housing insecurity, including overcrowding or lack of stable housing.
    'z_HE_SE_nat',  # Safety Environment: Reflects crime rates and safety conditions in the area, impacting mental and physical health.
    'z_HE_HE_nat',  # Healthcare Access: Indicates availability of healthcare facilities and services in the community.
    
    # Socioeconomic Domain
    'z_SE_EI_nat',  # Employment and Income: Measures employment rates and income stability within the community.
    'z_SE_EO_nat',  # Economic Opportunity: Captures opportunities for upward economic mobility.
    'z_SE_ER_nat',  # Educational Resources: Availability of resources supporting education, such as libraries and technology.
    'z_SE_HQ_nat',  # Housing Quality: Reflects the condition and quality of housing in the area.
    'z_SE_SR_nat',  # Social Resources: Measures community support systems and social cohesion.
    'z_SE_WL_nat'   # Workforce Longevity: Stability and longevity of employment within the workforce.
])

#load MIMIC Synthesized admissions table (COMPLEX_DIAGNOSIS + Zipcode)
admissions_data = pd.read_csv("ADMISSIONS_SYNTHETIC.csv")

#load MIMIC patients data 
patients_data = pd.read_csv("PATIENTS.csv")

reduce ADMISSIONS_SYNTHETIC

In [3]:
admissions_reduced = admissions_data[[
    'HADM_ID', 'SUBJECT_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 
    'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 
    'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 
    'ETHNICITY', 'DIAGNOSIS', 'HAS_CHARTEVENTS_DATA', 'COMPLEX_DIAGNOSIS',
    'Level','county_name','zip_code'
]]

connect patients with admissions by SUBJECT_ID

In [4]:
# connect patients table with ADMISSIONS_SYNTHETIC to generate new table 
patients_admissions = pd.merge(patients_data, admissions_reduced, on='SUBJECT_ID', how='inner')

patients_admissions.to_csv('Patients_Admissions.csv', index=False)

Remove Duplicates from patients_admissions

In [5]:
patients_admissions = patients_admissions.drop_duplicates()

patients_admissions.to_csv('Patients_Admissions.csv', index=False)

Check for duplicates, aggregate the data (mean of each score per county)

In [ ]:
# Check for duplicates in patients_admissions
patients_admissions_duplicates = patients_admissions[patients_admissions.duplicated(subset=['HADM_ID', 'county_name'])]
print(f"Duplicates in patients_admissions: {len(patients_admissions_duplicates)}")

# Remove duplicates, keeping the first occurrence
patients_admissions = patients_admissions.drop_duplicates(subset=['HADM_ID', 'county_name'])

# Check for duplicates in COI data
coi_duplicates = coi_data[coi_data.duplicated(subset=['county_name'])]
print(f"Duplicates in COI data: {len(coi_duplicates)}")


# Group by 'county_name' and aggregate numeric columns
"""
COI data has multiple rows for the same county_name
Aggregation combines multiple rows into one row per county by summarizing numeric columns into single values 
procedure: it took the mean of those scores for each county_name to produce row per county
"""
numeric_columns = coi_data.select_dtypes(include=['number']).columns
coi_data_grouped = coi_data.groupby('county_name')[numeric_columns].mean().reset_index()

# Retain only necessary columns for merging
coi_data_cleaned = coi_data[['county_name']].drop_duplicates().merge(
    coi_data_grouped, on='county_name', how='left'
)   

Duplicates in patients_admissions: 0
Duplicates in COI data: 722855


Perform Merge

In [8]:
# Merge patients_admissions with cleaned COI data
patients_admissions_coi = pd.merge(patients_admissions, coi_data_cleaned, on='county_name', how='inner')

# Save the merged dataset
patients_admissions_coi.to_csv('Patients_Admissions_COI.csv', index=False)